In [ ]:
import os
import json
import pandas as pd
import re

# Define the columns for the validation template
columns = [
    "Filename",
    "Name",
    "Email",
    "Phone",
    "Location",
    "Summary",
    "Education Degree",
    "Education Institution",
    "Education Year",
    "Experience Title",
    "Experience Company",
    "Experience Duration",
    "Experience Location",
    "Experience Description",
    "Skills",
    "Project Title",
    "Project Description",
    "Certifications",
    "Languages",
    "Social Profiles"
]

# Directory containing the JSON files
directory = "/home/shtlp_0152/Desktop/project 1/data-ingestion-pipeline/data/standardized_resumes"

# Get all filenames in the directory and sort them alphabetically
filenames = sorted([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.endswith(".json")])

# Create a DataFrame to store the results
data = {col: [""] * len(filenames) for col in columns}  # Initialize all columns with empty strings
data["Filename"] = filenames  # Populate the "Filename" column with actual filenames

# Function to validate email
def validate_email(email):
    email_pattern = re.compile(r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$')
    return bool(email_pattern.match(email))


def validate_phone_number(phone_number):
    phone_pattern = re.compile(r'''
        ^                                # start
        (\(?\+?\d{1,3}\)?[-.\u2013\s]*)? # optional country code, allowing – as sep
        (\(?\d{2,5}\)?[-.\u2013\s]*)?    # optional area code, allowing – as sep
        (                                # main number: either…
            \d{5}[-.\u2013\s]*\d{5}      #   5+5 format
        |                              # OR
            \d{3}[-.\u2013\s]*\d{3}[-.\u2013\s]*\d{4}  # 3-3-4 format
        )
        $                                # end
    ''', re.VERBOSE)
    phone_numbers = re.split(r'\s*[,/]\s*', phone_number)
    return all(phone_pattern.match(num.strip()) for num in phone_numbers)
# Iterate through each file and check for missing or empty fields
for i, filename in enumerate(filenames):
    file_path = os.path.join(directory, filename)
    try:
        with open(file_path, 'r') as file:
            json_data = json.load(file)
            
            # Map JSON fields to the corresponding columns
            data["Name"][i] = "PRESENT" if json_data.get("name") else "NULL"
            
            # Validate email
            email = json_data.get("email", "")
            if email:
                data["Email"][i] = "PRESENT" if validate_email(email) else "INVALID PRESENT"
            else:
                data["Email"][i] = "NULL"
            
            # Validate phone
            phone = json_data.get("phone", "")
            if phone:
                data["Phone"][i] = "PRESENT" if validate_phone_number(phone) else "INVALID PRESENT"
            else:
                data["Phone"][i] = "NULL"
            
            data["Location"][i] = "PRESENT" if json_data.get("location") else "NULL"
            data["Summary"][i] = "PRESENT" if json_data.get("summary") else "NULL"
            
            # Check education details
            if "education" in json_data and json_data["education"]:
                education = json_data["education"][0]  # Take the first education entry
                data["Education Degree"][i] = "PRESENT" if education.get("degree") else "NULL"
                data["Education Institution"][i] = "PRESENT" if education.get("institution") else "NULL"
                data["Education Year"][i] = "PRESENT" if education.get("year") else "NULL"
            else:
                data["Education Degree"][i] = "NULL"
                data["Education Institution"][i] = "NULL"
                data["Education Year"][i] = "NULL"
            
            # Check experience details
            if "experience" in json_data and json_data["experience"]:
                experience = json_data["experience"][0]  # Take the first experience entry
                data["Experience Title"][i] = "PRESENT" if experience.get("title") else "NULL"
                data["Experience Company"][i] = "PRESENT" if experience.get("company") else "NULL"
                data["Experience Duration"][i] = "PRESENT" if experience.get("duration") else "NULL"
                data["Experience Location"][i] = "PRESENT" if experience.get("location") else "NULL"
                data["Experience Description"][i] = "PRESENT" if experience.get("description") else "NULL"
            else:
                data["Experience Title"][i] = "NULL"
                data["Experience Company"][i] = "NULL"
                data["Experience Duration"][i] = "NULL"
                data["Experience Location"][i] = "NULL"
                data["Experience Description"][i] = "NULL"
            
            # Check other fields
            data["Skills"][i] = "PRESENT" if json_data.get("skills") else "NULL"
            if "projects" in json_data and json_data["projects"]:
                project = json_data["projects"][0]  # Take the first project entry
                data["Project Title"][i] = "PRESENT" if project.get("title") else "NULL"
                data["Project Description"][i] = "PRESENT" if project.get("description") else "NULL"
            else:
                data["Project Title"][i] = "NULL"
                data["Project Description"][i] = "NULL"
            
            data["Certifications"][i] = "PRESENT" if json_data.get("certifications") else "NULL"
            data["Languages"][i] = "PRESENT" if json_data.get("languages") else "NULL"
            data["Social Profiles"][i] = "PRESENT" if json_data.get("social_profiles") else "NULL"
    except (json.JSONDecodeError, KeyError):
        # If there's an error reading the JSON file, mark all fields as "NULL"
        for col in columns[1:]:  # Skip "Filename"
            data[col][i] = "NULL"

# Create a DataFrame from the data dictionary
merged_df = pd.DataFrame(data)

# Save the DataFrame to an Excel file
output_file = "evaluation_with_validation.xlsx"
merged_df.to_excel(output_file, index=False)

print(f"Excel file '{output_file}' created successfully with validation results.")

In [ ]:
# Count total number of rows in the dataset
total_rows = len(merged_df)

# Count total null values
total_nulls = merged_df.apply(lambda col: col.isin(["NULL"]).sum()).sum()

# Count invalid phone numbers
invalid_phone_count = merged_df["Phone"].isin(["INVALID PRESENT"]).sum()

# Count invalid email addresses
invalid_email_count = merged_df["Email"].isin(["INVALID PRESENT"]).sum()

# Create a detailed null count for specific columns
specific_null_counts = {
    "Null Names": merged_df["Name"].isin(["NULL"]).sum(),
    "Null Phone Numbers": merged_df["Phone"].isin(["NULL"]).sum(),  
    "Null Email Addresses": merged_df["Email"].isin(["NULL"]).sum(), 
    "Null Locations": merged_df["Location"].isin(["NULL"]).sum(),
    "Null Summaries": merged_df["Summary"].isin(["NULL"]).sum(),
    "Null Skills": merged_df["Skills"].isin(["NULL"]).sum(),
    "Null Education Degrees": merged_df["Education Degree"].isin(["NULL"]).sum(),
    "Null Education Institutions": merged_df["Education Institution"].isin(["NULL"]).sum(),
    "Null Education Years": merged_df["Education Year"].isin(["NULL"]).sum(),
    "Null Experience Titles": merged_df["Experience Title"].isin(["NULL"]).sum(),
    "Null Experience Companies": merged_df["Experience Company"].isin(["NULL"]).sum(),
    "Null Experience Durations": merged_df["Experience Duration"].isin(["NULL"]).sum(),
    "Null Experience Locations": merged_df["Experience Location"].isin(["NULL"]).sum(),
    "Null Experience Descriptions": merged_df["Experience Description"].isin(["NULL"]).sum(),
    "Null Project Titles": merged_df["Project Title"].isin(["NULL"]).sum(),
    "Null Project Descriptions": merged_df["Project Description"].isin(["NULL"]).sum(),
    "Null Certifications": merged_df["Certifications"].isin(["NULL"]).sum(),
    "Null Languages": merged_df["Languages"].isin(["NULL"]).sum(),
    "Null Social Profiles": merged_df["Social Profiles"].isin(["NULL"]).sum(),
}

# Combine all metrics into a summary DataFrame
report_data = {
    "Metric": [
        "Total Number of Entries",
        "Total Null Values",
        "Invalid Phone Numbers",
        "Invalid Email Addresses",
    ] + list(specific_null_counts.keys()),
    "Details": [
        total_rows,
        total_nulls,
        invalid_phone_count,
        invalid_email_count,
    ] + list(specific_null_counts.values()),
}
report_df = pd.DataFrame(report_data)

# Save the report to an Excel file
report_file = "data_quality_report.xlsx"
report_df.to_excel(report_file, index=False)

print(f"Detailed data quality report saved as '{report_file}'.")

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Read the Excel file
excel_file = "data_quality_report.xlsx"  # Replace with your actual file path
df = pd.read_excel(excel_file)

# Extract fields and null values from the Excel file
fields = df["Metric"].tolist()  # Assuming the column name is 'Metric'
null_values = df["Details"].tolist()  # Assuming the column name is 'Details'

# Remove "Total Null Values" from the data
if "Total Null Values" in fields:
    index = fields.index("Total Null Values")
    fields.pop(index)
    null_values.pop(index)

# Plot the bar graph
plt.figure(figsize=(12, 8)) 
bars = plt.barh(fields, null_values, color='skyblue') 
plt.xlabel('Number of Values')
plt.title('Values from Resume Field') 
plt.gca().invert_yaxis() 
plt.tight_layout()

# Add annotations to display the number of null values on each bar
for bar, value in zip(bars, null_values):
    plt.annotate(str(value), 
                 xy=(value, bar.get_y() + bar.get_height() / 2), 
                 xytext=(5, 0),  # Offset the text slightly
                 textcoords="offset points", 
                 va='center', 
                 ha='left', 
                 fontsize=10)

plt.savefig('values_chart.png')
plt.show()

In [8]:
def compute_score(row):
    score = 0
    for val in row[1:]:  # skip filename
        if val == "PRESENT":
            score += 1
        elif val == "INVALID PRESENT":
            score -= 1
    return score

df['Quality Score'] = df.apply(compute_score, axis=1)
top_resumes = df.sort_values(by="Quality Score", ascending=False)
top_resumes[['Filename', 'Quality Score']]


,Filename,Quality Score
92,Kunal Kumar CV.json,19
56,DEEPALI_JENA_RESUME_1Yr.json,18
33,Anuj Maurya.json,18
74,HarshYadav.json,18
146,Resume Kushagra Wadhwa.json,18
...,...,...
106,Mohd.Saleem K_ShorthillsAi Resume.json,6
86,Jitin Kumar Vats_SHT Resume 2.json,5
95,MANEESH KUMAR AGARWAL - Resume 1.json,5
174,Siddhant CV.json,3


In [ ]:
import os
import json
import shutil

def move_resumes_with_missing_name(source_directory, destination_directory):
    # Ensure the destination directory exists
    os.makedirs(destination_directory, exist_ok=True)

    # Iterate through all JSON files in the source directory
    for filename in os.listdir(source_directory):
        if filename.endswith(".json"):
            file_path = os.path.join(source_directory, filename)
            try:
                with open(file_path, 'r') as file:
                    json_data = json.load(file)
                    # Check if the "name" field is missing or empty
                    if not json_data.get("name"):
                        shutil.move(file_path, os.path.join(destination_directory, filename))
                        print(f"Moved: {filename}")
            except (json.JSONDecodeError, KeyError):
                print(f"Error reading file: {filename}")

if __name__ == "__main__":
    source_directory = "/home/shtlp_0152/Desktop/project 1/data-ingestion-pipeline/data/standardized_resumes"
    destination_directory = "/home/shtlp_0152/Desktop/project 1/data-ingestion-pipeline/data/missing_name_resumes"
    move_resumes_with_missing_name(source_directory, destination_directory)
